In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle  

In [2]:
#load the datset 

data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#preprocess data and drop irrelevnat feature

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#we will encode geography and gender as they are categorical values 
# will encode geography usoing LabelEncoder and geography using one hot encoding

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])


In [5]:
# now using OHE for converting geography

from sklearn.preprocessing import OneHotEncoder
ohe_geo=OneHotEncoder()
geo_encoder=ohe_geo.fit_transform(data[['Geography']])
#now we wil convert this intoi df and merge with original dataframe also

geo_encoded_df=pd.DataFrame(geo_encoder.toarray(),columns=ohe_geo.get_feature_names_out(["Geography"]))
#giving array as data and feature names as columns

geo_encoded_df



,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
#now merging with data frame

data=pd.concat([data.drop("Geography",axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
#split data into features and target

X=data.drop("EstimatedSalary",axis=1)   #independent features
Y=data["EstimatedSalary"] 

In [8]:
#splitting into training and testing data

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

#scale the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [9]:
#saving encoders and sscaler filr

with open('label_encoder_gender.pkl','wb') as file:
  pickle.dump(label_encoder_gender,file)

with open('ohe_geo.pkl','wb') as file:
  pickle.dump(ohe_geo,file)

#saving scaler filr

with open('scaler.pkl','wb') as file:
  pickle.dump(scaler,file)

#TRAIN ANN WITH REGRESSION STATEMENT

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
#BUILD ANN MODEL

model=Sequential([
  Dense(64,activation='relu',input_shape=(X_train.shape[1],)) , # connecting first hidden layer with input
  Dense(32,activation='relu'),
  Dense(1) #output layer for regression , here default func applied i.e linear which is for regression only
]
)

In [20]:
#compile the model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
#setup tensoreboard

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%M%D-%H%M%S")

tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [22]:
# srtup earlystopping

earlystopping_callback=EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [23]:
#Train the model

history=model.fit(X_train,Y_train,
                  validation_data=(X_test,Y_test),
                  epochs=100,
                  callbacks=[tensorboard_callback,earlystopping_callback])


Epoch 1/100


250/250 [==============================] - 3s 6ms/step - loss: 100392.3203 - mae: 100392.3203 - val_loss: 98571.6875 - val_mae: 98571.6875
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99815.2266 - mae: 99815.2266 - val_loss: 97379.4766 - val_mae: 97379.4766
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 97714.9688 - mae: 97714.9688 - val_loss: 94267.4375 - val_mae: 94267.4375
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 93505.4922 - mae: 93505.4922 - val_loss: 88936.6797 - val_mae: 88936.6797
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 87197.3359 - mae: 87197.3359 - val_loss: 81785.1172 - val_mae: 81785.1172
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 79338.2734 - mae: 79338.2734 - val_loss: 73685.7422 - val_mae: 73685.7422
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 71077.0547 - mae: 71077.054

In [24]:
#load tensorboard extension

%load_ext tensorboard

In [27]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 5784), started 0:00:11 ago. (Use '!kill 5784' to kill it.)

In [28]:
##evalute model for test data

test_loss,test_mae=model.evaluate(X_test,Y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50252.2070 - mae: 50252.2070
Test MAE:50252.20703125


In [30]:
model.save('regression_model.h5')